In [2]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 34.5 MB/s eta 0:00:00


In [16]:
import cv2
import mediapipe as mp
import json
import numpy as np

# MediaPipe initialization 
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5)

# Function to extract landmarks from a video frame using MediaPipe
def extract_landmarks(frame):
    results = holistic.process(frame)
    if results.pose_landmarks:
        landmarks = [[landmark.x, landmark.y, landmark.z] for landmark in results.pose_landmarks.landmark]
        return landmarks
    else:
        return None

# Function to process and save landmarks data to JSON file
def process_and_save_videos(video_paths, output_json_path, label):
    data = []

    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            landmarks = extract_landmarks(frame)
            if landmarks is not None:
                data.append((landmarks, label))

        cap.release()

    print(f"Processed {len(data)} frames for label {label}")

    with open(output_json_path, 'w') as json_file:
        json.dump(data, json_file)


# Example: Process and save positive and negative video datasets
positive_video_paths = ["/content/T1.mp4", "/content/T1.mp4","/content/T2.mp4","/content/T3.mp4","/content/T4.mp4","/content/T5.mp4","/content/T6.mp4","/content/T7.mp4","/content/T8.mp4","/content/T9.mp4","/content/T10.mp4"]
negative_video_paths = ["/content/N1.mp4","/content/N2.mp4","/content/N3.mp4","/content/N4.mp4","/content/N5.mp4","/content/N6.mp4",]

process_and_save_videos(positive_video_paths, "positive_data.json", label=1)
process_and_save_videos(negative_video_paths, "negative_data.json", label=0)


Processed 36 frames for label 1
Processed 79 frames for label 0


In [20]:
import json
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load data from JSON files
with open("/content/positive_data.json", 'r') as json_file:
    positive_data = json.load(json_file)

with open("/content/negative_data.json", 'r') as json_file:
    negative_data = json.load(json_file)

# Combine positive and negative data
all_data = positive_data + negative_data

# Separate features (landmarks) and labels
X = [item[0] for item in all_data]
y = [item[1] for item in all_data]

# Flatten the features array
X_flat = [item for sublist in X for item in sublist]

# Convert to NumPy array
X_flat_array = np.array(X_flat)

# Reshape X_flat_array to match the number of labels
X_flat_array = X_flat_array.reshape(len(y), -1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flat_array, y, test_size=0.2, random_state=42)

# Initialize and train the RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9130434782608695
